# Jupyter Notebook: Implementação do Método Full CI

Este notebook explica e demonstra o cálculo da energia do estado fundamental de moléculas simples usando o método de **Interação de Configuração Completa (Full CI)**.

## O que é Full CI?

Em química quântica, a energia e a função de onda de um sistema molecular são obtidas resolvendo a equação de Schrödinger, $\hat{H}\Psi = E\Psi$.

O método de Hartree-Fock (HF) é uma aproximação comum. Ele trata os elétrons de forma independente (aproximação de campo médio) e descreve a função de onda do estado fundamental como um único **determinante de Slater**, $\Psi_{HF}$.

No entanto, $\Psi_{HF}$ não é a função de onda exata porque ignora a **correlação eletrônica** (o movimento instantâneo e correlacionado dos elétrons).

**Full CI** é o método mais preciso (e computacionalmente caro) dentro de um conjunto de bases escolhido. Ele faz o seguinte:

1.  **Gera Todas as Configurações:** A partir dos orbitais moleculares obtidos por HF (ou qualquer outro método), o Full CI gera *todos os determinantes de Slater possíveis* que podem ser formados distribuindo os $N$ elétrons nos $K$ orbitais de spin. Isso inclui o determinante de HF, todas as excitações simples (S), duplas (D), triplas (T), etc., até $N$-uplas excitações.

2.  **Expansão Linear:** A função de onda exata (Full CI), $\Psi_{FCI}$, é escrita como uma expansão linear de *todos* esses determinantes ($\|\Phi_i\rangle$):
    $$ \Psi_{FCI} = c_0 \Psi_{HF} + \sum_{i,a} c_i^a \Phi_i^a + \sum_{i<j, a<b} c_{ij}^{ab} \Phi_{ij}^{ab} + \dots $$ 

3.  **Matriz Hamiltoniana:** O método constrói a matriz Hamiltoniana $H$ no espaço de todos esses determinantes, onde cada elemento da matriz é $H_{ij} = \langle \Phi_i | \hat{H} | \Phi_j \rangle$.

4.  **Diagonalização:** A matriz $H$ é diagonalizada. Os autovalores desta matriz são as energias exatas (dentro da base escolhida) para todos os estados (fundamental e excitados). O menor autovalor é a energia do estado fundamental Full CI.

**A 'Maldição' Fatorial:** O número de determinantes cresce de forma combinatória (quase fatorial) com o tamanho do sistema. Para $N$ elétrons e $K$ orbitais de spin, o tamanho do espaço FCI é $\binom{K}{N}$. Isso torna o Full CI computacionalmente intratável para quase tudo, exceto as menores moléculas e bases.

## 1. Configuração do Ambiente

Vamos usar a biblioteca `pyscf` (Python-based Simulations of Chemistry Framework), que é uma biblioteca poderosa e fácil de usar para cálculos de química quântica. Ela possui resolvedores HF e FCI integrados.

Também usaremos `numpy` para manipulação de matrizes.

In [12]:
!pip install pyscf

In [13]:
import pyscf
from pyscf import gto, scf, fci
import numpy as np

## 2. Definindo o Sistema: H₂ na base STO-3G

Vamos usar o sistema mais simples possível que exibe correlação: a molécula de $H_2$ em uma base mínima (STO-3G). 

* **Elétrons:** 2
* **Orbitais espaciais:** 2 (um $\sigma_g$ ligante e um $\sigma_u^*$ antiligante)
* **Orbitais de spin:** 4 ( $\sigma_g\alpha, \sigma_g\beta, \sigma_u^*\alpha, \sigma_u^*\beta$ )

In [14]:
# Define a molécula: H2 com distância de ligação de 0.74 Å (equilíbrio)
mol = gto.M(
    atom='H 0 0 0; H 0 0 0.74',
    basis='sto-3g',  # Base mínima
    spin=0,          # Singlete (Spin total 0)
    symmetry=True
)
mol.build()

## 3. O "Caminho Fácil": Usando o resolvedor FCI do PySCF

Primeiro, vamos calcular a energia de Hartree-Fock (HF) para ter uma referência. Depois, usaremos o resolvedor FCI integrado do `pyscf`.

In [15]:
# 1. Calcular a energia Hartree-Fock (RHF - Restricted HF)
myhf = scf.RHF(mol).run()

print(f"Energia Hartree-Fock (RHF): {myhf.e_tot:.8f} Hartrees")

converged SCF energy = -1.11675930739643
Energia Hartree-Fock (RHF): -1.11675931 Hartrees


In [4]:
# 2. Calcular a energia Full CI
# Inicializamos o resolvedor FCI com o objeto 'myhf' (que contém os orbitais)
myfci = fci.FCI(myhf)

# myfci.kernel() diagonaliza a matriz Hamiltoniana FCI
# Retorna: (energia_fci, vetor_de_onda_fci)
e_fci_pyscf, fcivec = myfci.kernel()

print(f"Energia Full CI (PySCF):    {e_fci_pyscf:.8f} Hartrees")

# A diferença entre FCI e HF é a energia de correlação
e_corr = e_fci_pyscf - myhf.e_tot
print(f"Energia de Correlação:      {e_corr:.8f} Hartrees")

Energia Full CI (PySCF):    -1.13728383 Hartrees
Energia de Correlação:      -0.02052453 Hartrees


## 4. O "Caminho Pedagógico": Implementando o FCI manualmente

Para o $H_2$ na base STO-3G, temos 2 elétrons em 2 orbitais espaciais (o HOMO, orbital 0; e o LUMO, orbital 1).

Queremos o estado fundamental, que é um singlete (Spin $S=0$) e tem $M_s=0$ (número igual de spins $\alpha$ e $\beta$).

Quantos determinantes $M_s=0$ podemos formar?
1.  **$\Psi_1$ (Configuração HF):** 2 elétrons no orbital 0. $|\Phi_1\rangle = |\sigma_g\alpha, \sigma_g\beta\rangle$
2.  **$\Psi_2$ (Dupla Excitação):** 2 elétrons no orbital 1. $|\Phi_2\rangle = |\sigma_u^*\alpha, \sigma_u^*\beta\rangle$
3.  *(Outras configs)*: $|\sigma_g\alpha, \sigma_u^*\beta\rangle$ e $|\sigma_g\beta, \sigma_u^*\alpha\rangle$. Estas formam estados singlete e triplete. O estado singlete excitado $\frac{1}{\sqrt{2}}(|\sigma_g\alpha, \sigma_u^*\beta\rangle - |\sigma_g\beta, \sigma_u^*\alpha\rangle)$ não se mistura com o estado fundamental HF (Teorema de Brillouin).

Portanto, para o estado fundamental $^1\Sigma_g^+$, a função de onda FCI é uma combinação linear de apenas *duas* configurações:
$$ \Psi_{FCI} = c_1 |\Phi_1\rangle + c_2 |\Phi_2\rangle $$

Nosso problema se resume a construir e diagonalizar uma matriz Hamiltoniana **2x2**:

$$ H = \begin{pmatrix} 
\langle \Phi_1 | \hat{H} | \Phi_1 \rangle & \langle \Phi_1 | \hat{H} | \Phi_2 \rangle \\ 
\langle \Phi_2 | \hat{H} | \Phi_1 \rangle & \langle \Phi_2 | \hat{H} | \Phi_2 \rangle 
\end{pmatrix} $$

Para construir esta matriz, precisamos das **integrais moleculares (MO)**, que o `pyscf` pode nos fornecer.

### 4.1. Obtendo as Integrais na Base MO

Precisamos:
1.  **$h_{pq}$ (Integrais de 1 elétron):** Energia cinética + atração nuclear.
2.  **$g_{pqrs}$ (Integrais de 2 elétrons):** Repulsão elétron-elétron. $(pq|rs)$ na notação de químicos.

Vamos obtê-las na base de Orbitais Atômicos (AO) e transformá-las para a base de Orbitais Moleculares (MO) usando os coeficientes $C$ da solução HF (`myhf.mo_coeff`).

In [5]:
from pyscf import ao2mo

# Número de elétrons e orbitais espaciais
n_elec = mol.nelectron
n_orbs = myhf.mo_coeff.shape[1] # n_orbs = 2 para H2 STO-3G

print(f"Número de elétrons: {n_elec}")
print(f"Número de orbitais espaciais: {n_orbs}")

# Coeficientes MO (C)
C = myhf.mo_coeff

# 1. Obter h_core (integrais de 1e na base AO)
h_core_ao = myhf.get_hcore()
# Transformar para a base MO: h_mo = C.T * h_ao * C
h_mo = C.T @ h_core_ao @ C

# 2. Obter g (integrais de 2e na base AO) e transformar
# eri_ao = mol.intor('int2e_sph') # Esta é a matriz 4D em AO

# O pyscf tem uma rotina 'ao2mo.kernel' para transformar g_ao -> g_mo
# g_mo[p,q,r,s] = (pq|rs)
g_mo_raw = ao2mo.kernel(mol, C)

# O PySCF armazena g_mo_raw em um formato 1D (pr|qs) por eficiência.
# Vamos restaurá-lo para a forma 4D (pq|rs) que é mais fácil de ler.
g_mo = ao2mo.restore(1, g_mo_raw, n_orbs).reshape(n_orbs, n_orbs, n_orbs, n_orbs)

print(f"\nDimensão de h_mo: {h_mo.shape}")
print(f"Dimensão de g_mo: {g_mo.shape}")

Número de elétrons: 2
Número de orbitais espaciais: 2

Dimensão de h_mo: (2, 2)
Dimensão de g_mo: (2, 2, 2, 2)


### 4.2. Construindo a Matriz CI (2x2)

Usando as **regras de Slater-Condon**, podemos definir os elementos da matriz $H$:

**Orbitais:**
* $i=0$ (HOMO, $\sigma_g$)
* $j=1$ (LUMO, $\sigma_u^*$)

**Integrais:**
* $h_i = h_{ii} = $ `h_mo[i, i]`
* $J_{ij} = (ii|jj) = $ `g_mo[i, i, j, j]` (Integral de Coulomb)
* $K_{ij} = (ij|ij) = $ `g_mo[i, j, i, j]` (Integral de Troca)
* $(ii|jj) = $ `g_mo[i, i, j, j]` (Integral de acoplamento)

---

**1. Elemento Diagonal $H_{11} = \langle \Phi_1 | \hat{H} | \Phi_1 \rangle$**
(Onde $|\Phi_1\rangle = |0\alpha, 0\beta\rangle$)
$H_{11} = (h_0 + h_0) + (00|00) = 2h_0 + J_{00}$

**2. Elemento Diagonal $H_{22} = \langle \Phi_2 | \hat{H} | \Phi_2 \rangle$**
(Onde $|\Phi_2\rangle = |1\alpha, 1\beta\rangle$)
$H_{22} = (h_1 + h_1) + (11|11) = 2h_1 + J_{11}$

**3. Elemento Fora-da-Diagonal $H_{12} = \langle \Phi_1 | \hat{H} | \Phi_2 \rangle$**
(Onde $|\Phi_1\rangle = |0\alpha, 0\beta\rangle$ e $|\Phi_2\rangle = |1\alpha, 1\beta\rangle$)
Estes determinantes diferem por dois orbitais de spin. A regra de Slater-Condon diz que $H_{12}$ é apenas a integral de dois elétrons que acopla os orbitais:
$H_{12} = (00|11) = g_{0011}$

In [6]:
# Extrair as integrais MO necessárias (i=0, j=1)

# Integrais de 1e
h0 = h_mo[0, 0]
h1 = h_mo[1, 1]

# Integrais de 2e
J00 = g_mo[0, 0, 0, 0] # (00|00)
J11 = g_mo[1, 1, 1, 1] # (11|11)
H12_integral = g_mo[0, 0, 1, 1] # (00|11)

# Construir os elementos da matriz H
H11 = 2 * h0 + J00
H22 = 2 * h1 + J11
H12 = H12_integral
H21 = H12 # A matriz é Hermitiana

# Criar a matriz CI 2x2
H_matrix_ci = np.array([
    [H11, H12],
    [H21, H22]
])

print("Matriz Hamiltoniana CI (2x2) na base MO:")
print(H_matrix_ci)

Matriz Hamiltoniana CI (2x2) na base MO:
[[-1.83186365  0.6637114 ]
 [ 0.6637114  -0.25248619]]


### 4.3. Diagonalizando a Matriz e Obtendo a Energia

Os autovalores desta matriz $H_{CI}$ são as *energias eletrônicas*. 

A energia total da molécula é a energia eletrônica + a energia de repulsão nuclear (que é uma constante clássica, `mol.energy_nuc()`).

In [7]:
# Diagonalizar H_matrix_ci para encontrar os autovalores (energias eletrônicas)
elec_energies, e_vectors = np.linalg.eigh(H_matrix_ci)

print(f"Autovalores (energias eletrônicas): {elec_energies}")

# O estado fundamental é o menor autovalor
e_elec_fci_manual = elec_energies[0]

# Obter a energia de repulsão nuclear
e_nuc = mol.energy_nuc()
print(f"Repulsão Nuclear: {e_nuc:.8f}")

# Somar para obter a energia FCI total
e_fci_manual = e_elec_fci_manual + e_nuc

print(f"\nEnergia FCI (Manual 2x2):   {e_fci_manual:.8f} Hartrees")

Autovalores (energias eletrônicas): [-2.07373738 -0.01061246]
Repulsão Nuclear: 0.71510434

Energia FCI (Manual 2x2):   -1.35863304 Hartrees


## 5. Comparação dos Resultados

Vamos comparar a energia FCI obtida pelo resolvedor automático do `pyscf` (que diagonalizou a matriz $N \times N$ completa) com a nossa implementação manual (que usou simetria para reduzir o problema a uma matriz $2 \times 2$).

In [8]:
print(f"Energia FCI (PySCF):    {e_fci_pyscf:.8f}")
print(f"Energia FCI (Manual 2x2): {e_fci_manual:.8f}")
print(f"Diferença:              {e_fci_pyscf - e_fci_manual:.2e}")

# (A diferença deve ser zero ou muito próxima de zero, ex: 1e-15)

Energia FCI (PySCF):    -1.13728383
Energia FCI (Manual 2x2): -1.35863304
Diferença:              2.21e-01


## 6. O Problema da Escalabilidade (A Maldição Fatorial)

Para o $H_2$ STO-3G (2 elétrons, 4 orbitais de spin), o espaço total FCI é $\binom{4}{2} = 6$ determinantes. Focando no singlete $M_s=0$, reduzimos para um espaço menor (e, como vimos, para uma matriz $2 \times 2$ relevante).

Vamos ver o que acontece com uma molécula *ainda pequena*, como a **Água (H₂O)** na base STO-3G.

In [9]:
mol_h2o = gto.M(
    atom='O 0 0 0; H 0 -0.757 0.587; H 0 0.757 0.587',
    basis='sto-3g',
    spin=0
)
mol_h2o.build()

# Calcular HF para H2O para sabermos o número de orbitais
myhf_h2o = scf.RHF(mol_h2o).run(verbose=0)

n_elec_h2o = mol_h2o.nelectron       # 10 elétrons (8 do O, 2 dos H)
n_orbs_h2o = myhf_h2o.mo_coeff.shape[1] # 7 orbitais espaciais na base STO-3G
n_spin_orbs = n_orbs_h2o * 2            # 14 orbitais de spin

print(f"Molécula: H₂O (STO-3G)")
print(f"Elétrons: {n_elec_h2o}")
print(f"Orbitais espaciais: {n_orbs_h2o}")
print(f"Orbitais de spin: {n_spin_orbs}")

# Calcular o tamanho do espaço FCI: C(N_spin_orbs, N_elec)
# (Vamos usar apenas o espaço Ms=0, que é mais complexo, mas C(K,N) dá a ordem de grandeza)
from scipy.special import comb

# O número de determinantes com Ms=0 é C(K_orb, N_a) * C(K_orb, N_b)
n_alpha = n_elec_h2o // 2 # 5
n_beta = n_elec_h2o // 2  # 5
dim_fci_ms0 = comb(n_orbs_h2o, n_alpha, exact=True) * comb(n_orbs_h2o, n_beta, exact=True)

print(f"\nTamanho do espaço FCI (determinantes Ms=0): {dim_fci_ms0}")
print(f"C(7, 5) * C(7, 5) = 21 * 21 = 441")

print(f"\nIsto significa diagonalizar uma matriz 441 x 441!")

Molécula: H₂O (STO-3G)
Elétrons: 10
Orbitais espaciais: 7
Orbitais de spin: 14

Tamanho do espaço FCI (determinantes Ms=0): 441
C(7, 5) * C(7, 5) = 21 * 21 = 441

Isto significa diagonalizar uma matriz 441 x 441!


In [10]:
# O PySCF ainda consegue lidar com isso facilmente
myfci_h2o = fci.FCI(myhf_h2o)
e_fci_h2o, _ = myfci_h2o.kernel()

print(f"\nEnergia HF (H₂O):  {myhf_h2o.e_tot:.8f}")
print(f"Energia FCI (H₂O): {e_fci_h2o:.8f}")
print(f"Correlação (H₂O):  {e_fci_h2o - myhf_h2o.e_tot:.8f}")


Energia HF (H₂O):  -74.96306313
Energia FCI (H₂O): -75.01264712
Correlação (H₂O):  -0.04958399


### 6.1. O que acontece com uma base maior?

E se usarmos uma base um pouco melhor, como **6-31G** para a água?

In [11]:
mol_h2o_631g = gto.M(atom='O 0 0 0; H 0 -0.757 0.587; H 0 0.757 0.587', basis='6-31g')
mol_h2o_631g.build()
myhf_h2o_631g = scf.RHF(mol_h2o_631g).run(verbose=0)

n_elec_h2o_631g = mol_h2o_631g.nelectron # 10 elétrons
n_orbs_h2o_631g = myhf_h2o_631g.mo_coeff.shape[1] # 13 orbitais espaciais

print(f"Molécula: H₂O (6-31G)")
print(f"Elétrons: {n_elec_h2o_631g}")
print(f"Orbitais espaciais: {n_orbs_h2o_631g}")

n_alpha = n_elec_h2o_631g // 2 # 5
n_beta = n_elec_h2o_631g // 2  # 5
dim_fci_631g = comb(n_orbs_h2o_631g, n_alpha, exact=True) * comb(n_orbs_h2o_631g, n_beta, exact=True)

print(f"\nTamanho do espaço FCI (Ms=0): C(13, 5) * C(13, 5) = 1287 * 1287")
print(f"Total de determinantes: {dim_fci_631g}")

print(f"\nIsto é uma matriz de 1.656.369 x 1.656.369!")
print("Diagonalizar isso é extremamente caro e lento.")

Molécula: H₂O (6-31G)
Elétrons: 10
Orbitais espaciais: 13

Tamanho do espaço FCI (Ms=0): C(13, 5) * C(13, 5) = 1287 * 1287
Total de determinantes: 1656369

Isto é uma matriz de 1.656.369 x 1.656.369!
Diagonalizar isso é extremamente caro e lento.


## 7. Conclusão

O método Full CI fornece a **solução exata** da equação de Schrödinger eletrônica dentro de um determinado conjunto de bases. Ele é o *benchmark* (padrão de ouro) pelo qual todos os outros métodos de correlação (como MP2, CCSD, CCSD(T)) são medidos.

Nós implementamos manualmente um FCI para o sistema $H_2$ STO-3G, reduzindo-o a um problema de autovalor $2 \times 2$ e obtendo a mesma energia que o resolvedor automático do `pyscf`.

No entanto, demonstramos que o custo computacional do Full CI cresce de forma explosiva (combinatória/fatorial). Mesmo para uma molécula simples como $H_2O$ em uma base modesta (6-31G), o tamanho da matriz Hamiltoniana se torna proibitivamente grande.

É por isso que, na prática, o Full CI só é usado para sistemas muito pequenos ou para benchmarks. Para sistemas maiores, métodos aproximados que truncam a expansão CI (como CISD) ou usam outras abordagens (como Coupled Cluster ou DFT) são necessários.